# Dados Abertos Servidores Públicos

Este arquivo tem como objetivo realizar uma anáilise sobre os dados abertos referentes ao funcionalismo público. A análise tem foco nos redimentos dos funcionários e na organização de cada área do serviço público.

## Coleta automatizada dos dados 

O governo federal disponibiliza em: https://www.portaltransparencia.gov.br/download-de-dados/servidores uma plataforma para que os cidadãos possam realizar o download dos dados de forma manual, selecionando  ano e mês do documento desejado. Para automatizar esta operação foi utilizado o seguinte código em python:


In [ ]:
import requests
import time

year=2018
for month in range(1, 12):
        start = time.time()
        if month < 10:  
            url= 'https://www.portaltransparencia.gov.br/download-de-dados/servidores/'+ str(year)+"0"+str(month)+'_Servidores'
        else:
            url= 'https://www.portaltransparencia.gov.br/download-de-dados/servidores/'+ str(year)+str(month)+'_Servidores'
        print(url)
        r = requests.get(url, verify = False)
        with open("Servidores"+str(year)+str(month)+".zip", "wb") as code:
            code.write(r.content)
        end = time.time()
        print(end - start)


O site possui um problema de certificados SSL inválidos, então para contornar esse problema foi necessário desativar a verificação HTTP. 

Como os links são estáticos e seguem um mesmo padrão, uma regra simples seguida de um metodo para obter o arquivo é o suficiente para realizarmos o download. Outro problema detectado é a falta do valor tamanho do arquivo nos headers. Assim não é possível acompanhar o progresso do download. 

## Desempacotamento dos arquivos

O código a seguir realiza o desempacotamento dos arquivo .zip obtidos na última etapa. Como neste caso o uso de disco não é um gargalo, o método utilizado foi decompactar todos os arquivos e selecionar os aderentes à analise posteriormente.


In [ ]:
from zipfile import ZipFile 

year=2018
    for month in range(1, 12):
        with ZipFile("Servidores"+str(year)+str(month)+".zip", "r") as objZip:
            objZip.extractall()
 

## Função para seleção de arquivo

Esta função tem como objetivo selecionar o arquivo (relacionado ao tipo de dado e a data) e também selecionar a classe de interesse dentro de cada arquivo.

In [ ]:
import pandas as pd

def select_file(year, month, file_type, target_class):
    if month < 10: 
        df= pd.read_csv(str(year)+"0"+str(month)+"_"+file_type+".csv",decimal=",", sep=";",encoding="iso-8859-15");
    else:
        df= pd.read_csv(str(year)+str(month)+"_"+file_type+".csv",decimal=",", sep=";",encoding="iso-8859-15");
    df=pd.DataFrame(df)
    return df[target_class]
    




## Análise perfil do Funcionário


In [ ]:
df_cadastro=select_file(2018,1,"Cadastro",['Id_SERVIDOR_PORTAL','ORGSUP_LOTACAO','ORG_LOTACAO'])

Podemos agora visualizar as proporções em número de funcionários por orgão superior.

In [ ]:
df_cadastro.head

In [ ]:
import plotly.graph_objects as go


graph=df_cadastro.groupby(['ORGSUP_LOTACAO']).size().to_frame('c').reset_index()


fig = go.Figure(data=[go.Pie(labels=graph.ORGSUP_LOTACAO, values=graph.c)])
fig.show()



Observa-se  que uma grande parte das instâncias está definida como "Sem informação". Podemos então filtrar estes dados para analisar os orgãos que estão inclusos nessa classe. 

In [ ]:
df_cadastro_aux=df_cadastro[df_cadastro['ORGSUP_LOTACAO']=='Sem informação']

graph=df_cadastro_aux.groupby(['ORG_LOTACAO']).size().to_frame('c').reset_index()
fig = go.Figure(data=[go.Pie(labels=graph.ORG_LOTACAO, values=graph.c)])
fig.show()



A maior parte das instâncias sem classificação de orgão superior estão relacionadas ao Ministério da Saúde. 
A definição encontrada no dicionário de dados para o arquivo do tipo cadastro nos diz: 
ÓRGÃO SUPERIOR - Unidade da Administração Direta que tenha entidades por ele supervisionadas. 
Fonte: Manual do SIAFI

Os resultados obtidos nos mostram as duas maiores áreas em número de funcionários e o perfil de administrão pública nos quais os funcionários estão inseridos. Com a área da educação sendo submetida a adiminstração direta e a área da saúde sendo submetida à administração indireta do governo federal. Essa difrenciação nos regimes de administração podem ter impacto no perfil econômico do funcionário público.


## Análise de Remuneração

Seguimos para a próxima anáilise, esta agora referente à remuneração recebida pelos funcionários públicos.
        

In [ ]:
remuneracao_df=select_file(2018,1,"Remuneracao",['Id_SERVIDOR_PORTAL',"REMUNERAÇÃO BÁSICA BRUTA (R$)",'REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (R$)'])

Para fazer a distinção entre remuneração de diferentes orgãos públicos devemos realizar o merge entre a tabela de cadastro e tabela de remunerção.

In [ ]:
merged_df = pd.merge(df_cadastro, remuneracao_df, on='Id_SERVIDOR_PORTAL')

In [ ]:
merged_df

In [ ]:

import plotly.express as px

fig = px.box(merged_df, x="REMUNERAÇÃO BÁSICA BRUTA (R$)" , y='ORGSUP_LOTACAO' ,orientation='h' ,title='Remuneração  Grupo Superior')
fig.show()


O boxplot acima mostra a distribuição das remunerações nos diferentes orgãos de governo. Pode-se notar a grande presença de outliers. A maior mediana salarial está no ministério da Economica com aproximadamente 20 mil reais e o a menor mediana no Ministério da Previdência Social com aproximadamente 2 mil reais. O Ministério da Educação que possui o maior número de servidores possui mediana salarial 7,9 mil reais. As medianas indicam o valor que representa o meio da distribuição, no caso do ministério da Previdência social, por exemplo, 50% dos funcionários recebem menos que 2 mil reais de remuneração.  

In [ ]:
dfaux=merged_df[merged_df['ORGSUP_LOTACAO']=='Sem informação']
dfaux=dfaux[dfaux['ORG_LOTACAO']=='Ministério da Saúde']


In [ ]:
fig = px.box(dfaux, y="REMUNERAÇÃO BÁSICA BRUTA (R$)" , x='ORG_LOTACAO' ,title='Remuneração  para o Ministério da Saúde')
fig.show()



Por estar contido na categoria Sem informação, o Ministério da Saúde foi separado em uma segunda visualização. Assim como nos outros casos há uma grande presença de outliers. Com mediana salarial ficando em torno de 5.9 mil reais.

### Deduções


In [ ]:
merged_df["Deduções"]=merged_df["REMUNERAÇÃO BÁSICA BRUTA (R$)"] -merged_df['REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (R$)']

In [ ]:
descontos_df = merged_df.groupby(['ORGSUP_LOTACAO']).mean().reset_index()
descontos_df.sort_values('Deduções')



A tabela acima nos mostra as médias salariais e os descontos sobre estes salários. O Ministério do Esporte conta apresenta a maior média de descontos com valor aproximado de 4.2 mil reais. Isso pode indicar a alta aderência dos funcionários do ministério do Esporte a serviços que tenham desconto diretamente em folha, como crédito consignado, por exemplo.

## Conclusão

Esta breve análise mostrou a estrutura e remuneração da administração pública. Os maiores setores são respectivamente, Educação e Saúde. Os dados mostram que os funcionários ligados à adiministração indireta do Ministério da Saúde os salários mais representativos, são menores que os ligados ao Ministério da Educação.

